In [22]:
using MLJ
using DataFrames
using LinearAlgebra

In [3]:
using Plots

In [4]:
include("/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fp_cs361/gp_main.jl")

Table{AbstractVector{Continuous}}


fpred (generic function with 1 method)

In [5]:
include("/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fp_cs361/samples.jl")


save_samples (generic function with 2 methods)

In [6]:
include("/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fp_cs361/pso.jl")

Table{AbstractVector{Continuous}}


test_opt (generic function with 1 method)

# Data Conversion

In [7]:
# y = convert(Array{Float64,1}, y./10e9);

# # convert X from matrix to table 
# Xtable = MLJ.table(X./10e9)
# println(scitype(Xtable))
# schema(Xtable);

# GaussianProcessRegressor

In [8]:
# name = "GaussianProcessRegressor"
filter(model) = model.name == "GaussianProcessRegressor" &&
                model.package_name == "ScikitLearn"
models(filter)[1];

In [9]:
GPModel = @load GaussianProcessRegressor pkg="ScikitLearn" verbosity=0



MLJScikitLearnInterface.GaussianProcessRegressor

In [10]:
gpmodel = GPModel()

GaussianProcessRegressor(
    kernel = nothing,
    alpha = 1.0e-10,
    optimizer = "fmin_l_bfgs_b",
    n_restarts_optimizer = 0,
    normalize_y = false,
    copy_X_train = true,
    random_state = nothing) @182

In [11]:
r = range(gpmodel, :alpha, lower=10e-12, upper=10e-5, scale=:log);

In [12]:
self_tuning_gpmodel = TunedModel(model=gpmodel,
							  resampling=CV(nfolds=3),
							  tuning=Grid(resolution=10),
							  range=r,
							  measure=rms);

In [13]:
mach = machine(self_tuning_gpmodel, Xtable, y);
fit!(mach, verbosity=0)

Machine{DeterministicTunedModel{Grid,…},…} @387 trained 1 time; caches data
  args: 
    1:	Source @719 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @648 ⏎ `AbstractVector{Continuous}`


In [14]:
fitted_params(mach).best_model

GaussianProcessRegressor(
    kernel = nothing,
    alpha = 5.99484250318941e-11,
    optimizer = "fmin_l_bfgs_b",
    n_restarts_optimizer = 0,
    normalize_y = false,
    copy_X_train = true,
    random_state = nothing) @181

In [15]:
# did not impact rmse much :( 
entry = report(mach).best_history_entry

(model = GaussianProcessRegressor @181,
 measure = [RootMeanSquaredError @409],
 measurement = [1.3311284496454001],
 per_fold = [[1.25477238415913, 1.258683132794373, 1.4686632982193895]],)

In [16]:
function fpred(X)
    return MLJ.predict(mach, X)
end


fpred (generic function with 1 method)

# Try decision tree regressor for better rmse

In [23]:
Tree = @load DecisionTreeRegressor pkg=DecisionTree verbosity=0;


ErrorException: Failed to precompile MLJDecisionTreeInterface [c6f25543-311c-4c74-83dc-3ea6d1015661] to /Users/julietnwagwuume-ezeoke/.julia/compiled/v1.7/MLJDecisionTreeInterface/jl_Rt08C5.

In [ ]:
tree = Tree()

# Sensitivity Analysis

In [17]:
# # generate grid of samples 
# function samples_full_factorial(a,b,m)
#     ranges = [range]
# end

In [18]:
# forward / backward differentiation to calculate the indices 